## **Preprocesamiento**

Instalar y cargar librerías

In [21]:
! pip install plotly
! pip install scikit-learn

     ---------------------------------------- 15.2/15.2 MB 3.5 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.impute import SimpleImputer 

Funciones

In [70]:
#Función para calcular cantidad y porcentaje de nulos por variable
def nulos(x):
    cant_nulos=x.isnull().sum() 
    porc_nulos=round(x.isnull().sum().sum()/x.size, 3)*100
    return cant_nulos, porc_nulos

Lectura de datos

In [97]:
df_employee= pd.read_csv('./data/employee_survey_data.csv')
df_general=pd.read_csv('./data/general_data.csv', sep=";")
df_manager=pd.read_csv('./data/manager_survey_data.csv')
df_retirement=pd.read_csv('./data/retirement_info.csv', sep=";")

##### Exploración y limpieza de datos

##### BD employee_survey

In [98]:
df_employee.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [99]:
df_employee.shape
#la BD employee_survey contiene 4 variables, las cuales son el ID del empleado, nivel de satisfación del ambiente laboral, 
#nivel de satisfación laboral, Balance de la vida laboral y personal. 

# En total se tienen 4410 registros de empleados.

(4410, 4)

In [100]:
df_employee.info()
# Cantidad de datos no nulos y tipo de dato de cada variable

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 137.9 KB


In [101]:
# Convertir variables tipo númericas a categóricas
df_employee=df_employee.astype({'EnvironmentSatisfaction': object, 'JobSatisfaction': object, 'WorkLifeBalance':object})

In [102]:
df_employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   EmployeeID               4410 non-null   int64 
 1   EnvironmentSatisfaction  4385 non-null   object
 2   JobSatisfaction          4390 non-null   object
 3   WorkLifeBalance          4372 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.9+ KB


In [103]:
df_employee.apply(nulos)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,0.0,25.0,20.0,38.0
1,0.0,0.6,0.5,0.9


In [104]:
#Dado que se tiene un % de datos nulos muy pequeño para cada varible, se opta por aplicar la estrategia de tratamiento de nulos ffill
# ffill(): copiar el valor previo en la celda donde encuentre un valor nulo
df_employee = df_employee.ffill()

In [105]:
df_employee.apply(nulos)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0


In [106]:
#Verificar los valores únicos de cada variable
for variable in list(df_employee.columns[1:]):
  y = df_employee[variable].unique()
  print(variable)
  print(y)

EnvironmentSatisfaction
[3. 2. 4. 1.]
JobSatisfaction
[4. 2. 1. 3.]
WorkLifeBalance
[2. 4. 1. 3.]


##### BD general_data

In [107]:
df_general.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,...,4.0,Y,12,8,2,9.0,2,6,0,4


In [108]:
df_general.shape
#la BD employee_survey contiene 23 variables

# En total se tienen 4410 registros de empleados.

(4410, 23)

In [109]:
#Cantidad de no nulos y tipo de dato por variable
df_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   BusinessTravel           4410 non-null   object 
 2   Department               4410 non-null   object 
 3   DistanceFromHome         4410 non-null   int64  
 4   Education                4410 non-null   int64  
 5   EducationField           4410 non-null   object 
 6   EmployeeCount            4410 non-null   int64  
 7   EmployeeID               4410 non-null   int64  
 8   Gender                   4410 non-null   object 
 9   JobLevel                 4410 non-null   int64  
 10  JobRole                  4410 non-null   object 
 11  MaritalStatus            4410 non-null   object 
 12  MonthlyIncome            4410 non-null   int64  
 13  NumCompaniesWorked       4391 non-null   float64
 14  Over18                  

In [112]:
#Verificar los valores únicos de cada variable
for variable in list(df_general.columns[1:]):
  y = df_general[variable].unique()
  print(variable)
  print(y)

BusinessTravel
['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department
['Sales' 'Research & Development' 'Human Resources']
DistanceFromHome
[ 6 10 17  2  8 11 18  1  7 28 14  3  4 16  9  5 20 29 15 13 24 19 22 25
 21 26 27 12 23]
Education
[2 1 4 5 3]
EducationField
['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
EmployeeID
[   1    2    3 ... 4408 4409 4410]
Gender
['Female' 'Male']
JobLevel
[1 4 3 2 5]
JobRole
['Healthcare Representative' 'Research Scientist' 'Sales Executive'
 'Human Resources' 'Research Director' 'Laboratory Technician'
 'Manufacturing Director' 'Sales Representative' 'Manager']
MaritalStatus
['Married' 'Single' 'Divorced']
MonthlyIncome
[131160  41890 193280 ...  37020  23980  54680]
NumCompaniesWorked
[ 1.  0.  3.  4.  2.  7.  9.  5.  6.  8. nan]
PercentSalaryHike
[11 23 15 12 13 20 22 21 17 14 16 18 19 24 25]
StockOptionLevel
[0 1 3 2]
TotalWorkingYears
[ 1.  6.  5. 13.  9. 28. 10. 21. 16. 37.  7.  3. 15.  8. nan 12. 

In [111]:
#Se detectó que las variables 'EmployeeCount','Over18','StandardHours' están compuestas por un valor único, por tanto, se procede a eliminarlas
df_general=df_general.drop(['EmployeeCount','Over18','StandardHours'], axis=1)

In [115]:
#Aplicar función nulos para identificar la cantidad y porcentaje de estos por cada variable
df_general.apply(nulos)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.2,0.0,0.0,0.0,0.0


In [116]:
# Para las variables de 'TotalWorkingYears' y 'NumCompaniesWorked' se detectaron nulos pero en una porpoción pequeña (inferior del 10%)
#Definir la forma de imputar nulos
imputar = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
# imputar datos faltantes
df_general.iloc[:,:] = imputar.fit_transform(df_general.iloc[:,:]) 

In [119]:
# Convertir variables tipo númericas a categóricas
df_general=df_general.astype({'Education': object, 'JobLevel': object, 'StockOptionLevel':object})

In [120]:
# Convertir variables tipo float a enteras
df_general=df_general.astype({'NumCompaniesWorked': int})

In [125]:
#las variables tipo string convertirlas a minúsculas
df_general['BusinessTravel']=df_general['BusinessTravel'].str.lower()
df_general['Department']=df_general['Department'].str.lower()
df_general['EducationField']=df_general['EducationField'].str.lower()
df_general['Gender']=df_general['Gender'].str.lower()
df_general['JobRole']=df_general['JobRole'].str.lower()
df_general['MaritalStatus']=df_general['MaritalStatus'].str.lower()